In [29]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import re
import random

## Import Data

In [30]:
raw = pd.read_csv('./Task_Data.csv')
raw.rename(columns = {'number_project' : 'projects', 'satisfaction_level': 'satisfaction', 'last_evaluation': 'evaluation', 'time_spend_company' : 'years', 'sales' : 'department', 'promotion_last_5years' : 'promotion', 'average_montly_hours' : 'hours', 'time_at_company' : 'years', 'Work_accident' : 'accident'}, inplace = True)
raw.head()

,satisfaction,evaluation,projects,hours,years,accident,left,promotion,department,salary
0,0.38,0.53,2,157,3,0,1,0,sales,low
1,0.80,0.86,5,262,6,0,1,0,sales,medium
2,0.11,0.88,7,272,4,0,1,0,sales,medium
3,0.72,0.87,5,223,5,0,1,0,sales,low
4,0.37,0.52,2,159,3,0,1,0,sales,low


## Clean Data

#### Check for null values

In [31]:
raw.loc[raw.isnull().any(axis = 1)]

,satisfaction,evaluation,projects,hours,years,accident,left,promotion,department,salary


#### Check for potential Outliers

In [32]:
col = 'hours'

mean = raw[col].mean()
std_dev = raw[col].std()

low = mean - std_dev*3
print('There are', len(raw.loc[raw[col] <= low]), 'low outliers in the', col, 'column')


high = mean + std_dev*3
print('There are', len(raw.loc[raw[col] >= high]), 'high outliers in the', col, 'column')

There are 0 low outliers in the hours column
There are 0 high outliers in the hours column


# 1) High Performing Employees

In [33]:
# Define high performing employees as having a last_evaluation > 75 percentile

high_perf = raw.loc[raw['evaluation'] >= raw.evaluation.quantile(0.75)].reset_index(drop = True)
high_perf.head()

,satisfaction,evaluation,projects,hours,years,accident,left,promotion,department,salary
0,0.11,0.88,7,272,4,0,1,0,sales,medium
1,0.72,0.87,5,223,5,0,1,0,sales,low
2,0.89,1.00,5,224,5,0,1,0,sales,low
3,0.84,0.92,4,234,5,0,1,0,sales,low
4,0.78,0.99,4,255,6,0,1,0,sales,low


### 1a) Salary - Discrete

In [34]:
high_perf.groupby(['salary', 'left']).count()['department'].to_frame('count').reset_index()

# Indicates salary as corelation

,salary,left,count
0,high,0,226
1,high,1,15
2,low,0,1189
3,low,1,754
4,medium,0,1199
5,medium,1,462


In [35]:
salaries = ['low', 'medium', 'high']
sal_left_percent = []


for salary in salaries:
    sal_left = len(high_perf.loc[(high_perf['salary'] == salary) & (high_perf['left'] == 1)])
    sal_total = len(high_perf.loc[high_perf['salary'] == salary])
    sal_left_percent.append(round(sal_left/sal_total*100))
    

salary = pd.DataFrame(data = {'salary' : salaries, 'left_percentage' : sal_left_percent})
salary

,salary,left_percentage
0,low,39
1,medium,28
2,high,6


### 1b) Promotions - Discrete

In [36]:
high_perf.groupby(['promotion', 'left']).count()['department'].to_frame('count').reset_index()

# Indicates promotions as corelation

,promotion,left,count
0,0,0,2552
1,0,1,1230
2,1,0,62
3,1,1,1


In [37]:
promotion = ['not_promoted', 'promoted']
prom_left_percent = []
i = 0

while i <= 1:
    prom_left = len(high_perf.loc[(high_perf['promotion'] == i) & (high_perf['left'] == 1)])
    prom_total = len(high_perf.loc[high_perf['promotion'] == i])
    prom_left_percent.append(round(prom_left/prom_total*100))
    i += 1

promotion = pd.DataFrame(data = {'promotion' : promotion, 'left_percentage' : prom_left_percent})
promotion

,promotion,left_percentage
0,not_promoted,33
1,promoted,2


### 1c) Accidents - Discrete

In [38]:
high_perf.groupby(['accident', 'left']).count()['department'].to_frame('count').reset_index()

# shows employees more likely to stay following an accident
# This is unusual and not deos not lead to any actionable retention improvements for the client

,accident,left,count
0,0,0,2154
1,0,1,1178
2,1,0,460
3,1,1,53


### 1d) Continuous data (to be analysed in Tableau)

In [51]:
high_perf['proj_per_year'] = high_perf['projects'] / high_perf['years']
high_perf.groupby(['left']).mean()[['hours', 'satisfaction', 'years', 'proj_per_year']].reset_index()

high_perf_cont = high_perf[['satisfaction', 'projects', 'hours', 'years', 'proj_per_year', 'left']]
high_perf_cont

,satisfaction,projects,hours,years,proj_per_year,left
0,0.11,7,272,4,1.750000,1
1,0.72,5,223,5,1.000000,1
2,0.89,5,224,5,1.000000,1
3,0.84,4,234,5,0.800000,1
4,0.78,4,255,6,0.666667,1
...,...,...,...,...,...,...
3840,0.76,5,238,5,1.000000,1
3841,0.73,5,162,4,1.250000,1
3842,0.91,5,254,5,1.000000,1
3843,0.89,5,228,5,1.000000,1


## Export Data for Visualisation

In [53]:
salary.to_csv('salary.csv')
promotion.to_csv('promotion.csv')
high_perf_cont.to_csv('continuous.csv')

# Add continuous fields

# 2) Predictive Modelling

### Vectorise salary fields

In [41]:
def sal_convert(sal):
    if sal == 'low':
        return 1
    elif sal == 'medium':
        return 2
    else:
        return 3

raw['salary'] = raw['salary'].apply(lambda x: sal_convert(x))

### Create Fields

In [42]:
x = raw.drop(['left', 'department'], axis = 1).reset_index(drop = True)
y = raw['left'].astype('str').reset_index(drop = True)
y

0        1
1        1
2        1
3        1
4        1
        ..
14994    1
14995    1
14996    1
14997    1
14998    1
Name: left, Length: 14999, dtype: object

### Fit classifier

In [43]:
from sklearn.model_selection import train_test_split

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size = 0.2, random_state = 42)

In [44]:
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(penalty = 'l1', C = 0.01, solver = 'liblinear', max_iter = 500)

lr.fit(x_train, y_train)

LogisticRegression(C=0.01, max_iter=500, penalty='l1', solver='liblinear')

### Gridsearch

In [45]:
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RepeatedStratifiedKFold

#grid={'C':[0.01,0.1,10,100,1000], 'solver': ['newton-cg', 'lbfgs', 'liblinear'],
      #'penalty':['none','l1','l2','elasticnet']}

#clf=GridSearchCV(estimator=LogisticRegression(),param_grid=grid,n_jobs = 1,scoring='accuracy')

#result = clf.fit(x_train,y_train)

In [46]:
#print(result.best_params_)

### Score and predictions

In [47]:
# Reset index to compare predictions to actual result
x_test = x_test.reset_index(drop = True)
y_test = y_test.reset_index(drop = True)

print('The predictions score is', lr.score(x_test, y_test))

The predictions score is 0.775


In [48]:
i = 500

print('The prediction for employee', i, 'is', lr.predict(x_test)[i])
print('The probability of employee', i, 'leaving is', lr.predict_proba(x_test)[i][1])

The prediction for employee 500 is 0
The probability of employee 500 leaving is 0.0822146369916852


### 2b) Employees at risk of leaving

In [105]:
new_df = pd.concat([x_train, x_test]).reset_index(drop=True)
new_df['left'] = pd.concat([y_test, y_train]).reset_index(drop=True)

train_risk = lr.predict_proba(x_train)[:,1]
test_risk = lr.predict_proba(x_test)[:,1]

In [110]:
risk = np.append(train_risk, test_risk)
new_df['risk'] = risk

In [130]:
# For current employees use left = '0', for previous employees use left = '1'

risk_level = 0.2

new_df.loc[(new_df['risk'] >= risk_level) & (new_df['left'] == '0')]

,satisfaction,evaluation,projects,hours,years,accident,promotion,salary,left,risk
1,0.16,0.78,4,196,5,0,0,3,0,0.407009
4,0.11,0.93,7,284,4,0,0,1,0,0.621061
5,0.09,0.83,6,295,5,0,0,1,0,0.722127
8,0.31,0.49,4,165,3,1,0,1,0,0.238142
9,0.44,0.50,2,148,3,0,0,2,0,0.253774
...,...,...,...,...,...,...,...,...,...,...
14986,0.10,0.85,7,259,4,0,0,1,0,0.598075
14987,0.37,0.49,2,151,3,0,0,3,0,0.209550
14990,0.10,0.85,6,273,4,0,0,2,0,0.536737
14994,0.61,0.74,2,143,6,0,0,2,0,0.277670
